In [1]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import json
import torch 

In [2]:
import sys 
sys.path.append("../")
from src.model import load_model
from src.utils import test_to_tensor

%load_ext autoreload
%autoreload 2

In [3]:
MODEL_DIR = "C:/Users/yjyang027/Documents/git/Multimodal_22/models/"
print(os.listdir(MODEL_DIR))

['cite_multitask_L2optim.th', 'multi_ncorr_optim.th']


In [4]:
DATA_DIR = "C:/Users/yjyang027/Documents/git/Multimodal_22/data/"

SUBMISSION = os.path.join(DATA_DIR,"sample_submission.csv")
EVALUATION_IDS = os.path.join(DATA_DIR,"evaluation_ids.csv")

CITE_TEST_INPUTS = os.path.join(DATA_DIR,"cite_test_x.h5ad") 
MULTIOME_TEST_INPUTS = os.path.join(DATA_DIR,"multi_test_x.h5ad")

In [5]:
# submit: map eval to submission by "row_id"

eval_ids = pd.read_csv(EVALUATION_IDS, index_col="row_id")
eval_ids

,cell_id,gene_id
row_id,,
0,c2150f55becb,CD86
1,c2150f55becb,CD274
2,c2150f55becb,CD270
3,c2150f55becb,CD155
4,c2150f55becb,CD112
...,...,...
65744175,2c53aa67933d,ENSG00000134419
65744176,2c53aa67933d,ENSG00000186862
65744177,2c53aa67933d,ENSG00000170959


In [6]:
eval_ids[48663*140-2:48663*140+2]

,cell_id,gene_id
row_id,,
6812818,ad5a949989b2,CD88
6812819,ad5a949989b2,CD224
6812820,8d287040728a,ENSG00000204091
6812821,8d287040728a,ENSG00000198938


In [ ]:
# # confirm the order

# len(np.unique(eval_ids["cell_id"])) # 65443

# print(ada_x.obs.tail())
# print(eval_ids["cell_id"][48663*140-5:48663*140+2])

# ada_y = sc.read_h5ad(os.path.join(DATA_DIR, "cite_train_y.h5ad"))
# print(ada_y.var[:1], ada_y.var[-1:])
# print(eval_ids["gene_id"][140-2:140+2])

### map preds to submit

In [ ]:
# eval_ids_cite = eval_ids[:48663*140]
# eval_ids_multi = eval_ids[48663*140:]

# print(len(eval_ids_cite), len(eval_ids_multi))

In [ ]:
# with open(os.path.join(DATA_DIR,'feature_id_dict_cite.json'), 'r') as f:
#     feature_id_dict_cite = json.load(f)
# with open(os.path.join(DATA_DIR,'feature_id_dict_multi.json'), 'r') as f:
#     feature_id_dict_multi = json.load(f)
# with open(os.path.join(DATA_DIR,'cell_id_dict_cite.json'), 'r') as f:
#     cell_id_dict_cite = json.load(f)
# with open(os.path.join(DATA_DIR,'cell_id_dict_multi.json'), 'r') as f:
#     cell_id_dict_multi = json.load(f)

# print(len(cell_id_dict_cite), len(feature_id_dict_cite))
# print(len(cell_id_dict_multi), len(feature_id_dict_multi))

In [ ]:
# # generate test idx (cell, gene)
# eval_ids = pd.read_csv(EVALUATION_IDS, index_col="row_id")
# eval_ids_cite = eval_ids[:48663*140]
# eval_ids_multi = eval_ids[48663*140:]
# print(len(eval_ids_cite), len(eval_ids_multi))

# eval_ids_cite["cell_idx"] = eval_ids_cite["cell_id"].map(cell_id_dict_cite)
# eval_ids_cite["feature_idx"] = eval_ids_cite["gene_id"].map(feature_id_dict_cite)
# eval_ids_cite["submit_idx"] = eval_ids_cite[["cell_idx", "feature_idx"]].values.tolist()
# eval_ids_multi["cell_idx"] = eval_ids_multi["cell_id"].map(cell_id_dict_multi)
# eval_ids_multi["feature_idx"] = eval_ids_multi["gene_id"].map(feature_id_dict_multi)
# eval_ids_multi["submit_idx"] = eval_ids_multi[["cell_idx", "feature_idx"]].values.tolist()
# np.savez_compressed('submit_idx_cite.npz', np.array(list(eval_ids_cite["submit_idx"])))
# np.savez_compressed('submit_idx_multi.npz', np.array(list(eval_ids_multi["submit_idx"])))

In [10]:
# load test idx

eval_ids_cite = np.load(os.path.join(DATA_DIR, "submit_idx_cite.npz"))
eval_ids_multi = np.load(os.path.join(DATA_DIR, "submit_idx_multi.npz"))
eval_ids_cite = eval_ids_cite.f.arr_0
eval_ids_multi = eval_ids_multi.f.arr_0

print(eval_ids_cite.shape, eval_ids_multi.shape)

(6812820, 2) (58931360, 2)


In [8]:
# load preds

pred_y_cite = np.load('../preds/pred_y_cite.npz')
pred_y_multi = np.load('../preds/pred_y_multi.npz')
pred_y_cite = pred_y_cite.f.arr_0
pred_y_multi = pred_y_multi.f.arr_0

print(pred_y_cite.shape, pred_y_multi.shape)

(48663, 140) (55935, 23418)


In [22]:
# extract preds in order of eval

pred_y_cite_submit = [pred_y_cite[eval_ids_cite[i][0]][eval_ids_cite[i][1]] for i in range(len(eval_ids_cite))]
pred_y_multi_submit = [pred_y_multi[eval_ids_multi[i][0]][eval_ids_multi[i][1]] for i in range(len(eval_ids_multi))]

print(len(pred_y_cite_submit), len(pred_y_multi_submit))

6812820 58931360


In [23]:
submission = pd.read_csv(SUBMISSION, index_col="row_id")
submission.head()

,target
row_id,
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [25]:
assert len(submission) == len(pred_y_cite_submit) + len(pred_y_multi_submit)

# submission.loc[submission.index[:len(pred_y.ravel())], "target"] = pred_y.ravel() # assign
submission.loc[submission.index[:len(pred_y_cite_submit)], "target"] = pred_y_cite_submit
submission.loc[submission.index[len(pred_y_cite_submit):], "target"] = pred_y_multi_submit
submission[48663*140-2:48663*140+2]

,target
row_id,
6812818,8.223215
6812819,3.807193
6812820,-0.046130
6812821,0.475898


In [29]:
OUTPUT = os.path.join(DATA_DIR,"submission_1.zip")

submission.to_csv(OUTPUT, index=True, compression="zip")